In [ ]:
import scanpy as sc ###
import dynamo as dyn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
import squidpy as sq ###
import sys
import time
import os
import re
from pandas.core.frame import DataFrame

In [ ]:
inh5ad="../Adult_5d1_C01830E6F6_FB_assays.h5ad"
outpath="./Adult_5d1_C01830E6F6"

In [ ]:
anndata = sc.read(inh5ad)
anndata.obsm['X_spatial'] = anndata.obsm['spatial'].copy()

In [ ]:
rotated_coords = anndata.obsm['X_spatial'].copy()
rotated_coords[:, 1] = -rotated_coords[:, 1]
rotated_coords

In [ ]:
anndata.obsm['X_spatial'] = rotated_coords

In [ ]:
co_all = {"FB_fascia_1_Adult":'#db2400',"FB_inflammatory_Adult":"#63FFAC","FB_oxidative_stress_Adult":'#FFFF52',"FB_myo_Adult":"#1CE6FF"}

In [ ]:
cell_list = ["FB_oxidative_stress_Adult","FB_myo_Adult"]
anndata = anndata[anndata.obs['celltype'].isin(cell_list)]
print(anndata.obs['celltype'].value_counts())

co = {celltype: co_all[celltype] for celltype in cell_list if celltype in co_all}

In [ ]:
if not os.path.exists(outpath):
    # 创建新的文件夹
    os.makedirs(outpath,exist_ok=True)
    print("文件夹已创建")
else:
    print("文件夹已存在")

os.chdir(outpath)

In [ ]:
sc.pp.calculate_qc_metrics(anndata, expr_type='unspliced', layer = "unspliced", inplace = True)

In [ ]:
## 过滤细胞，基因；sct，pca
preprocessor = dyn.pp.Preprocessor(cell_cycle_score_enable=True)
preprocessor.preprocess_adata(anndata, recipe='monocle')
dyn.tl.reduceDimension(anndata, enforce = True)
#dyn.pl.umap(anndata, color='celltype',show_legend= 'on data',alpha=1,background='black',pointsize=0.25,figsize=(8, 7), color_key=co) #alpha-透明度

In [ ]:
## 计算高维空间的速度向量
dyn.tl.dynamics(anndata,  cores=10)

In [ ]:
# 识别这些基因、评估速度估计的置信度
dyn.tl.gene_wise_confidence(anndata, group='celltype',lineage_dict = {"FB_oxidative_stress_Adult" :["FB_myo_Adult"]} )

In [ ]:
dyn.tl.reduceDimension(anndata, enforce = True)

# Project high dimensional velocity vectors onto given low dimensional embeddings, and/or compute cell transition
dyn.tl.cell_velocities(anndata, method="fp", basis='spatial',enforce=True,  transition_genes = list(anndata.var_names[anndata.var.use_for_pca])) #先将 RNA 速度投影到 spatial 空间
# 检查细胞速度的置信度
dyn.tl.cell_wise_confidence(anndata)
dyn.tl.confident_cell_velocities(anndata, group='celltype', lineage_dict={"FB_oxidative_stress_Adult" :["FB_myo_Adult"]},only_transition_genes=True,basis='spatial')

In [ ]:
dyn.vf.VectorField(anndata,
                   basis='spatial',
                   M=100)

dyn.vf.rank_velocity_genes(anndata,
                           groups='celltype',
                           vkey="velocity_S")

dyn.vf.acceleration(anndata, basis='spatial')
dyn.ext.ddhodge(anndata, basis='spatial')
transition = DataFrame(anndata.var_names[anndata.var.use_for_transition])
transition.columns = ['gene']
transition_genes = transition['gene'].tolist()

In [ ]:
dyn.pl.streamline_plot(anndata, color=['celltype'],
                       basis='spatial',
                       show_legend='None',
                       figsize=(8, 4),
                       color_key=co,
                       pointsize=0.05,
                       background='black',
                       cmap ='viridis',
                       show_arrowed_spines=True,
                       calpha=1,
                       dpi=300,
                       linewidth=0.5,
                       s_kwargs_dict = {"alpha": 0.8},
                       save_show_or_return='save',
                       save_kwargs = {"path":"./Adult_5d1_C01830E6F6/","prefix":"celltype.spatial"}
                       #streamline_color = "#ffffff"
                      )

In [ ]:
dyn.pl.streamline_plot(anndata, color=['spatial_ddhodge_potential'],
                       basis='spatial',
                       show_legend='None',
                       figsize=(8, 4),
                       color_key=co,
                       pointsize=0.05,
                       background='black',
                       cmap ='viridis',
                       show_arrowed_spines=True,
                       calpha=1,
                       dpi=300,
                       linewidth=0.5,
                       s_kwargs_dict = {"alpha": 0.8},
                       save_show_or_return='save',
                       save_kwargs = {"path":"./Adult_5d1_C01830E6F6/","prefix":"spatial_ddhodge_potential"}
                       #streamline_color = "#ffffff"
                      )